In [1]:
import pandas as pd
import sys
sys.path.append('./utils')
from utils import *
from rank import *
import japanize_matplotlib
from process import * 
import tqdm

In [2]:
# データ保存先
data_folda = '../data/'

In [3]:
pseudo_time_series = pd.read_pickle(data_folda+'pseudo_time_series_with_v_google.pkl')

* V_GOOGLE の変数を追加

In [4]:
pd_sf = read_table(data_folda,"V_RETTY",columns="all",read_snowflake=False,col="*")

In [5]:
import rank
# general rank を与える
ranker=rank.Ranker()
pd_sf=ranker.add_general_rank(pd_sf)
r=pd_sf["general_rank"].value_counts().sort_index()
#S,A,B,L,F,N の順に value_countes を並べる
r=r.loc[["S","A","B","L","F","N"]]
display(r)
# cummulative sumを表示
display(pd.concat([r,r.cumsum()],axis=1))

/home/ec2-user/restaurant_scoring_ftdata/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/restaurant_scoring_ftdata/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/restaurant_scoring_ftdata/pg/./utils/rank.py:78: SettingWithCop

S    184517
A    258213
B     10454
L    191495
F        60
N        86
Name: general_rank, dtype: int64

,general_rank,general_rank
S,184517,184517
A,258213,442730
B,10454,453184
L,191495,644679
F,60,644739
N,86,644825


In [6]:
pd_sf.rename(columns={"general_rank":"general_rank_RETTY"},inplace=True)

In [7]:
# info.isOfficial の空文字を0, trueを1に変換
pd_sf["infos.isOfficial"]=pd_sf["infos.isOfficial"].replace({"":0,"true":1}).astype(int)

# infos.onlineReservation falseを0, trueを1に変換
pd_sf["infos.onlineReservation"]=pd_sf["infos.onlineReservation"].replace({"false":0,"true":1}).astype(int)
# infos.access.transferTime1 徒歩X分のXを取り出す。intに変換
pd_sf["infos.access.transferTime1"]=pd_sf["infos.access.transferTime1"].str.extract("(\d+)").astype(float)
# infos.numberOfSeats.valueの "席"を取り除く
pd_sf["infos.numberOfSeats.value"]=pd_sf["infos.numberOfSeats.value"].str.replace("席","")
#空文字はNaNに変換にしてfloatに変換
pd_sf["infos.numberOfSeats.value"]=pd_sf["infos.numberOfSeats.value"].replace("",np.nan)
pd_sf["infos.numberOfSeats.value"]=pd_sf["infos.numberOfSeats.value"].astype(float)

# infos.updateInfo.firstReviewDate の空文字をNaNに変換後、datetimeに変換
pd_sf["infos.updateInfo.firstReviewDate"]=pd_sf["infos.updateInfo.firstReviewDate"].replace("",np.nan)
pd_sf["infos.updateInfo.firstReviewDate"]=pd.to_datetime(pd_sf["infos.updateInfo.firstReviewDate"],format="%Y年%m月%d日")


# pd_sf["infos.wantToGo"]の空文字をNaNに変換後、floatに変換
pd_sf["infos.wantToGo"]=pd_sf["infos.wantToGo"].replace("",np.nan).astype(float)
# pd_sf["infos.went"]の空文字をNaNに変換後、floatに変換
pd_sf["infos.went"]=pd_sf["infos.went"].replace("",np.nan).astype(float)
# infos.rateByWentの%を取り除いて、空文字をNaNに変換後、floatに変換
pd_sf["infos.rateByWent"]=pd_sf["infos.rateByWent"].str.replace("%","").replace("",np.nan).astype(float)

# infos.excellentの空文字をNaNに変換後、floatに変換
pd_sf["infos.excellent"]=pd_sf["infos.excellent"].replace("",np.nan).astype(float)
pd_sf[["infos.good","infos.average"]]=pd_sf[["infos.good","infos.average"]].replace("",np.nan).astype(float)

# infos.onlineReservation falseを0, trueを1に変換
pd_sf["infos.coupon"]=pd_sf["infos.coupon"].replace({"false":0,"true":1}).astype(int)

# float_listをfloatに変換
float_list=['infos.familiar.users.count',"infos.photoCount.Photograph","infos.photoCount.Cooking","infos.photoCount.Interior",
          "infos.photoCount.Exterior","infos.photoCount.Menu","infos.reviewCount.Review",
          "infos.reviewCount.Lunch","infos.reviewCount.Dinner",
          "menus.course_list_count","menus.dishes_list_count","menus.lunch_list_count",
          "menus.drink_list_count","menus.takeout_list_count"]
for col in float_list:
    pd_sf[col]=pd_sf[col].replace("",np.nan).astype(float)

In [8]:
add_list=["restaurant_id",'infos.isOfficial', 'infos.familiar.users.genge',
       'infos.familiar.users.count', 'infos.familiar.stars',
       'general_rank_RETTY', 'infos.reserveEntireStore',
       'infos.onlineReservation', 'infos.access.transferTime1',
       'infos.numberOfSeats.value', 'infos.counterSeats.value',
       'infos.privateRoom.value', 'infos.updateInfo.firstReviewDate',
       'infos.wantToGo', 'infos.updateInfo.lastReviewDate',
       'infos.went', 'infos.rateByWent', 'infos.excellent', 'infos.coupon',
       'infos.photoCount.Photograph', 'infos.photoCount.Cooking',
       'infos.photoCount.Interior', 'infos.photoCount.Exterior',
       'infos.photoCount.Menu', 'infos.reviewCount.Review',
       'infos.reviewCount.Lunch', 'infos.reviewCount.Dinner',
       'menus.course_list_count',
       'menus.dishes_list_count', 'menus.lunch_list_count',
       'menus.drink_list_count', 'menus.takeout_list_count']
          
          
          

In [9]:
pseudo_time_series=pd.merge(pseudo_time_series,pd_sf[add_list],on="restaurant_id",how="left")

In [ ]:
pseudo_time_series.to_pickle(data_folda+'pseudo_time_series_with_v_retty.pkl')

In [ ]:
pseudo_time_series=pd.read_pickle(data_folda+'pseudo_time_series_with_v_retty.pkl')

In [ ]:
pseudo_time_series.columns

Index(['restaurant_id', 'original_id', 'closed_date', 'business_status',
       'prefecture', 'base_date', 'closed_flg', 'target3', 'target6',
       'eval_period', 'id', 'name', 'branch_name', 'opened_date', 'created_at',
       'genres', 'latitude', 'longitude', 'net_reservation_flg',
       'price_range_lunch_by_owner', 'price_range_dinner_by_owner',
       'price_range_lunch_by_review', 'price_range_dinner_by_review',
       'coupons_count', 'reviews_count', 'hozon_count', 'all_photo_count',
       'food_photo_count', 'drink_photo_count', 'interior_photo_count',
       'exterior_photo_count', 'real_opened_date', 'openning_days',
       'price_range_lunch_by_owner_num', 'price_range_dinner_by_owner_num',
       'price_range_lunch_by_review_num', 'price_range_dinner_by_review_num',
       'overview.aggregateRating', 'overview.reviewCount',
       'general_rank_GOOGLE', 'infos.isOfficial', 'infos.familiar.users.genge',
       'infos.familiar.users.count', 'infos.familiar.stars',
     

In [ ]:
check_list=['infos.onlineReservation', 'infos.access.transferTime1',
       'infos.numberOfSeats.value', 'infos.counterSeats.value',
       'infos.privateRoom.value', 'infos.updateInfo.firstReviewDate',
       'infos.wantToGo', 'infos.updateInfo.lastReviewDate', 'infos.went',
       'infos.rateByWent', 'infos.excellent', 'infos.coupon',
       'infos.photoCount.Photograph', 'infos.photoCount.Cooking',
       'infos.photoCount.Interior', 'infos.photoCount.Exterior',
       'infos.photoCount.Menu', 'infos.reviewCount.Review',
       'infos.reviewCount.Lunch', 'infos.reviewCount.Dinner',
       'menus.course_list_count', 'menus.dishes_list_count',
       'menus.lunch_list_count', 'menus.drink_list_count',
       'menus.takeout_list_count']

In [ ]:
by_close_flg=pseudo_time_series.groupby("closed_flg")[check_list].agg(["mean","min","max","median"]).T

/tmp/ipykernel_6001/2668809178.py:1: FutureWarning: ['infos.counterSeats.value', 'infos.privateRoom.value', 'infos.updateInfo.lastReviewDate'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  by_close_flg=pseudo_time_series.groupby("closed_flg")[check_list].agg(["mean","min","max","median"]).T


* net reservation（食べログ） と online reservation (retty) の比較 

In [ ]:
pseudo_time_series[["infos.onlineReservation","net_reservation_flg"]].value_counts()
#食べログで resevation flag が立っているても、半分以上は retty で reservation flag が立っていない

NameError: name 'pseudo_time_series' is not defined

* prefecture ごとの closed_flg の割合

In [ ]:
# 47都道府県でloop
for pref in pseudo_time_series["prefecture"].unique():
    print(pref)
    display(pseudo_time_series[pseudo_time_series["prefecture"]==pref]["closed_flg"].value_counts(normalize=True))
    print("")

静岡県


0    0.992182
1    0.007818
Name: closed_flg, dtype: float64


大阪府


0    0.986165
1    0.013835
Name: closed_flg, dtype: float64


愛知県


...

0    0.995865
1    0.004135
Name: closed_flg, dtype: float64


佐賀県


0    0.993635
1    0.006365
Name: closed_flg, dtype: float64


鳥取県


0    0.991979
1    0.008021
Name: closed_flg, dtype: float64